# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [143]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [144]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [145]:
# Import the CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [164]:
# Define Panel Visualization Functions
sfo_sort_neighborhoods = sfo_data.groupby('neighborhood').mean()
expensive_neighborhoods = sfo_sort_neighborhoods.sort_values("sale_price_sqr_foot", ascending=False).head(10)

locations_average = df_neighborhood_locations.groupby('Neighborhood').mean()
neighborhood_averages = (
    pd.concat([locations_average, sfo_sort_neighborhoods], axis=1, sort=True).dropna().reset_index()
)



In [147]:
def housing_units_per_year():
    housing_units_per_year=sfo_data[["housing_units"]].groupby([sfo_data.index]).sum()
    housing_units_per_year=housing_units_per_year.hvplot()
    return housing_units_per_year


In [148]:
def average_gross_rent():
    average_gross_rent = sfo_data[["gross_rent"]].groupby([sfo_data.index]).mean()
    average_gross_rent = average_gross_rent.hvplot(title='Average Gross Rents')
    return average_gross_rent

In [149]:
    """Average Sales Price Per Year."""
def average_sales_price():
        average_sales_price = sfo_data[["sale_price_sqr_foot"]].groupby([sfo_data.index]).mean()
        average_sales_price = average_sales_price.hvplot(title='Average Sales Price')
        return average_sales_price

In [150]:
    """Average Prices by Neighborhood."""
def average_price_by_neighborhood():
        average_price_by_neighborhood = sfo_data.groupby(['year', 'neighborhood']).mean()
        average_price_by_neighborhood= average_price_by_neighborhood.hvplot(x='year', y='sale_price_sqr_foot', groupby='neighborhood')
        return average_price_by_neighborhood

In [151]:
    """Top 10 Most Expensive Neighborhoods."""
def top_most_expensive_neighborhoods():
        top_most_expensive_neighborhoods = expensive_neighborhoods.hvplot.bar('neighborhood', 'sale_price_sqr_foot', ylim=(500, 1000),width = 500, rot=90)
        return top_most_expensive_neighborhoods

In [152]:
    """Parallel Coordinates Plot."""
def parallel_coordinates():
        parallel_coordinates= px.parallel_coordinates(expensive_neighborhoods)
        return parallel_coordinates

In [156]:
def parallel_categories():
        parallel_categories = px.parallel_categories(expensive_neighborhoods)
        return parallel_categories 

In [165]:
    """Neighborhood Map"""
def neighborhood_map():
        neighborhood_map = px.scatter_mapbox(
        neighborhood_averages,
        lat = "Lat",
        lon = "Lon",
        color = "index")

        return neighborhood_map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [166]:
housing_column = pn.Column(
    "## Housing Information", 
    housing_units_per_year(),
    average_gross_rent()
)


sales_column = pn.Column(
    "## Sales and Neighborhood Data",
    average_sales_price(),
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods()
)

parallel_column = pn.Column(
   "## Parallel Plots",
  parallel_coordinates(),
    parallel_categories()
)
map_column = pn.Column(
    "## Neighborhood Map",
    neighborhood_map()
)

#Create tabs
SFO_dashboard = pn.Tabs(
    ("Housing Information", housing_column), ("Sales and Neighborhood Data", sales_column), 
    ("Parallel Plots", parallel_column), 
   ("Neighborhood Map", map_column)
)


## Serve the Panel Dashboard

In [167]:
SFO_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [3] HoloViews(Bars)
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)